In [ ]:
import math
import sys

sys.path.append("../")

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np

from constants import SPEED_OF_SOUND

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

np.random.seed(1)

def custom_save(function=plt.savefig, fname=""):
    if not os.path.exists(os.path.dirname(fname)):
        os.makedirs(os.path.dirname(fname))
        print('created new directory', os.path.dirname(fname))
    try:
        # to make sure suptitle is not cut off
        function(fname, bbox_inches="tight")
    except:
        function(fname)
    print("saved as", fname)

from matplotlib import rcParams

rcParams["figure.max_open_warning"] = False

# 1. General setup

In [ ]:
# dimension = 2
# uniform = False
# gt_elevation = None

baseline = 0.11  # 0.115
distance = 1.925  # distance of source

# specific to experiment setup.
sources_2d = np.array([0, distance]).reshape((1, -1))
gt_middle = np.arctan2(sources_2d[0, 1], sources_2d[0, 0])

In [ ]:
from mic_array import get_square_array, ambiguity_test, get_min_distance
from plotting_beamform import plot_setup

mics_2d = get_square_array(baseline=baseline, delta=0)

filenames = [
    "back_left",  # 0, should be 2
    "back_right",  # 1, should be 0
    "front_left",  # 2, should be 1
    "front_right",  # 3, should be 3
]
plot_setup(mics_2d, sources_2d)
l = plt.gca().get_legend()
plt.legend(filenames, loc="upper left")
plt.gca().add_artist(l)

# 2. Signals

## 2.a Read signals from files

In [ ]:
import os
from file_parser import parameters, read_recordings
import pandas as pd


## we have only 0deg for low and normal
dir_names = [
    "recordings_9_7_20",  # audio shield measurements with buffer length 256, only 200Hz source
    "recordings_14_7_20",  # audio shield measurements with buffer length 8060, only 200Hz source
    "recordings_16_7_20",  # recordings with measurement mics, 200Hz and white_noise source
]
sources = [
    "200Hz",
    "white_noise",
]  # source types (single frequency at 200Hz or white noise)
loudnesses = ["high", "normal", "low"]  # loudness of source
gt_degrees_list = [0, 20, 40]  # rotation of drone with respect to source.

df = pd.DataFrame(
    columns=[
        "loudness",
        "gt_degrees",
        "dir_name",
        "source",
        "signals_props",
        "signals_source",
        "signals_all",
    ]
)
for loudness in loudnesses:
    for gt_degrees in gt_degrees_list:
        for dir_name in dir_names:
            for source in sources:
                try:
                    signals_props, signals_source, signals_all = read_recordings(
                        dir_name,
                        loudness=loudness,
                        gt_degrees=gt_degrees,
                        source=source,
                    )
                    df.loc[len(df), :] = dict(
                        loudness=loudness,
                        gt_degrees=gt_degrees,
                        dir_name=dir_name,
                        source=source,
                        signals_props=signals_props,
                        signals_source=signals_source,
                        signals_all=signals_all,
                    )
                except FileNotFoundError:
                    print("skipping", dir_name, loudness, gt_degrees, source)

## DEBUGGING: Signal Check

In [ ]:
from plotting_beamform import plot_sine_signals, normalize

loudness = "high"
gt_degrees = 40
source = "200Hz"
signals_props, signals_source, signals_all = read_recordings(
    dir_name, loudness=loudness, gt_degrees=gt_degrees, source=source
)
gt_azimuth = (gt_middle - gt_degrees) * np.pi / 180
print("ground truth angle", gt_azimuth * 180 / np.pi)
plot_sine_signals(signals_source, normalize_signal=True, length=200)

# 4. Algorithms

## 4.1 dataset

In [ ]:
from algos_beamforming import get_powers, get_beampattern
from algos_beamforming import get_lcmv_beamformer, get_das_beamformer
from algos_beamforming import get_autocorrelation

from plotting_beamform import *
import itertools

# selected_frequencies = "uniform"
# selected_frequencies = "single"
# selected_frequencies = "props"
# selected_frequencies = "notch"
# selected_frequencies = "middle"
selected_frequencies_list = [#"single", "uniform", #"harmonics",
                             "between", "between_snr"]
#selected_frequencies_list = ["between_snr_high"] #["between", "between_snr"]
#selected_frequencies_list = ["between_snr_high", "between", "between_snr"]
#selected_frequencies_list = ["between_snr_high", "single"]
#selected_frequencies_list = ["single"]

# selected_buffer = "source"  # source signal only
# selected_buffer = "all" # use all signals
# selected_buffer = "props"  # propeller signals only
#selected_buffer_list = ["source", "all"] #["props", "source", "all"]
selected_buffer_list = ["all"] #["props", "source", "all"]
#n_buffer = int(800 * 44100 / 32000)  # n_times//2 #256
n_buffer = 2**8  # n_times//2 #256
print('buffer size', n_buffer)

# select dataset
loudness_list = ["high"]#, "normal", "low"]

#source_list = ["200Hz"]
source_list = ["white_noise"]#, "200Hz"]

# source = "white_noise"
dir_name = "recordings_16_7_20"

In [ ]:
Fs = parameters[dir_name]["Fs"]
time_index = parameters[dir_name]["time_index"]
frequencies = np.fft.rfftfreq(n=n_buffer, d=1 / Fs)

min_dist = get_min_distance(mics_2d)
max_freq = 2 * SPEED_OF_SOUND / (2 * min_dist)
print('maximum frequency:', max_freq)

## DEBUGGING: test frequency selection schemes

In [ ]:
from algos_beamforming import select_frequencies

np.random.seed(3)

buffer = signals_all[:, time_index : time_index + n_buffer]
buffer_f = np.fft.rfft(buffer, axis=1)
all_frequencies = np.fft.rfftfreq(n=n_buffer, d=1 / Fs)

amp_ratio = 1  # amplitued selection ratio

#num_frequencies = None  # set to none to use all allowed
num_frequencies = 10  # set to number of randomly chosen frequencies

candidate_bins = select_frequencies(
    n_buffer, Fs, "between", 
    max_freq=max_freq,
    buffer_f=buffer_f, 
    num_frequencies=num_frequencies, delta=1
)
filtered_bins = select_frequencies(
    n_buffer,
    Fs,
    "between_snr",
    max_freq=max_freq,
    buffer_f=buffer_f,
    num_frequencies=num_frequencies,
    amp_ratio=amp_ratio, delta=1
)
uniform_bins = select_frequencies(
    n_buffer,
    Fs,
    "uniform",
    max_freq=max_freq,
    buffer_f=buffer_f,
    num_frequencies=num_frequencies, delta=1
)

candidate_frequencies = all_frequencies[candidate_bins]
filtered_frequencies = all_frequencies[filtered_bins]
uniform_frequencies = all_frequencies[uniform_bins]

for i in range(buffer.shape[0]):
    plt.loglog(all_frequencies, np.abs(buffer_f[i]), ls=":")
[plt.axvline(x=f, color="g") for f in candidate_frequencies]
[plt.axvline(x=f, color="b", ls=':') for f in filtered_frequencies]
[plt.axvline(x=f, color="r", ls=':') for f in uniform_frequencies]
plt.title("smart frequency selection")
avg_amp = np.mean(np.abs(buffer_f))
# assert abs(avg_amp - amp) < 1e-10, (avg_amp, amp)
plt.axhline(y=avg_amp, color="r", label="average amplitude")
plt.axhline(y=avg_amp * amp_ratio, color="k", ls=":", label="threshold")
plt.legend()

# [plt.axvline(x=f, color='r') for f in exclude_frequencies]
pass

## 4.2 Evaluate algorithms

In [ ]:
from signals import get_power

np.random.seed(1)

combinations = [np.sum, np.product]
thetas_scan = np.linspace(0, 360, 181) * np.pi / 180
method_names = ['MVDR', 'DAS']

for params in itertools.product(
        loudness_list,
        source_list,
        selected_buffer_list,
        selected_frequencies_list,
    ):
    
    ####### PARAMETER READING #########
    loudness, source, selected_buffer, selected_frequencies = params
    signals_degs = df.loc[(df.loudness == loudness)
                          & (df.source == source)
                          & (df.dir_name == dir_name), 
                          ["gt_degrees", "signals_source", "signals_all", "signals_props"], ]
    
    gt_degrees_list = signals_degs.gt_degrees.unique()
    print("found data for degrees:", gt_degrees_list)
    
    shifted_combination = {m: {c:None for c in combinations} for m in method_names}
    axs_combination = {c: plt.subplots(1, len(method_names)) for c in combinations}
    for gt_degrees, signals in signals_degs.groupby('gt_degrees'):
        out_dir = f"Output/{dir_name}/buffer{n_buffer}/{selected_buffer}/"

        if len(signals) == 0:
            print("skipping", params)
            continue

        signals_signal = signals["signals_source"].values[0]
        signals_noise = signals["signals_props"].values[0]
        snr = get_power(signals_signal) - get_power(signals_noise)

        signals_chosen = signals[f"signals_{selected_buffer}"].values[0]
        n_times = signals_chosen.shape[1]
        assert n_buffer <= n_times
        buffer = signals_chosen[:, time_index:time_index + n_buffer]

        buffer_f = np.fft.rfft(buffer, axis=1)
        frequency_bins = select_frequencies(n_buffer,
                                            Fs,
                                            selected_frequencies,
                                            max_freq=max_freq,
                                            buffer_f=buffer_f,
                                            num_frequencies=num_frequencies, delta=1)
        frequencies_hz = frequencies[frequency_bins]

        ####### BEAMFORMING #########
        Rx_sub = get_autocorrelation(buffer, frequency_bins)
        methods = {m: np.zeros((Rx_sub.shape[0], len(thetas_scan))) for m in method_names}

        for i, theta_scan in enumerate(thetas_scan):
            constraints = [(theta_scan, 1)]
            H_mvdr, conds, conds_big = get_lcmv_beamformer(Rx_sub, frequencies_hz, mics_2d, constraints)
            H_das = get_das_beamformer(theta_scan, frequencies_hz, mics_2d)

            # mvdr_scan[:, i] = get_beampattern(theta_scan, frequencies_hz, mics_2d, H)
            # das_scan[:, i] = get_beampattern(theta_scan, frequencies_hz, mics_2d, H_das)

            methods['MVDR'][:, i] = get_powers(H_mvdr, Rx_sub)
            methods['DAS'][:, i] = get_powers(H_das, Rx_sub)

        ###### PLOTTING #######
        title = f"{loudness}_{gt_degrees}deg_{source}_{selected_frequencies}"
        fig, ax = plot_signals(buffer, Fs, plot_frequencies=frequencies_hz)
        ax.set_title(f"{title}, {selected_buffer} signal, SNR:{snr:.2f}dB")
        fname = f"{out_dir}/signal_{loudness}_{gt_degrees}deg_{source}_{selected_frequencies}.png"
        custom_save(fname=fname)

        gt_azimuth = gt_middle - gt_degrees * np.pi / 180

        fig, axs = plt.subplots(2, 2, sharex=True, gridspec_kw={"height_ratios": [2, 1]})
        fig.set_size_inches(10, 5)
        for i, (method, scan) in enumerate(methods.items()):
            plot_spectrum_linear(scan, thetas_scan, frequencies_hz, ax=axs[0, i])
            axs[0, i].set_title(method)
            axs[1, i].set_xlabel("theta [deg]")

            for combination in combinations:
                normalized_combination = normalize(combination(scan, axis=0))
                axs[1, i].semilogy(
                    thetas_scan,
                    normalized_combination,
                    label=combination.__name__,
                )
            if not (selected_buffer == "props"):
                axs[1, i].axvline(x=gt_azimuth, color="k", ls=":")
                axs[0, i].axvline(x=gt_azimuth, color="k", ls=":")
        axs[0, 1].set_ylabel("")
        axs[0, 0].legend("", frameon=False)
        axs[0, 1].legend(bbox_to_anchor=[1.0, 1.0], loc="upper left")
        axs[1, 1].legend(bbox_to_anchor=[1.0, 1.0], loc="upper left")
        plt.tight_layout()

        title = f"{loudness}_{gt_degrees}deg_{source}_{selected_frequencies}"
        fname = f"{out_dir}/results_{title}.png"
        fig.suptitle(f"{title}, {selected_buffer} signal", y=1.01)
        custom_save(fig.savefig, fname)
        
        if len(gt_degrees_list) == 1:
            continue
        
        #### COMBINATINE THE RESULTS FROM DIFFERENT ANGLES #####
        for i, (method, scan) in enumerate(methods.items()):
            for combination in combinations:
                spectrum = normalize(combination(scan, axis=0))
                
                thetas_shifted = (np.unwrap(thetas_scan + (gt_degrees/180*np.pi)) + 2*np.pi) % (2*np.pi)
                approx_shift = np.argmin(thetas_shifted)

                spectrum_shifted = np.r_[spectrum[approx_shift:], spectrum[:approx_shift]]
                if shifted_combination[method][combination] is None:
                    shifted_combination[method][combination] = spectrum_shifted
                else:
                    shifted_combination[method][combination] = np.c_[shifted_combination[method][combination], spectrum_shifted]
        
    if len(gt_degrees_list) == 1:
        continue
                    
    for combination in combinations:
        fig, axs = axs_combination[combination] 
        fig.set_size_inches(10, 5)
        for i, (method, scan) in enumerate(methods.items()):
            responses = shifted_combination[method][combination] # 
            for j in range(responses.shape[1]): # different angles
                axs[i].semilogy(thetas_scan, responses[:, j])
            responses_combi = combination(responses, axis=1)
            axs[i].semilogy(thetas_scan, responses_combi, color='k')
            axs[i].set_title(f"{method}")
            axs[i].axvline(x=gt_middle, color='k', ls=":")
            
        title = f"{loudness}_{source}_{selected_frequencies}"
        fname = f"{out_dir}/{combination.__name__}_{title}.png"
        custom_save(fig.savefig, fname=fname)

In [ ]:
print(selected_frequencies)
len(frequencies)
max_bin = int(max_freq * n_buffer // Fs)
frequency_bins = select_frequencies(n_buffer, Fs,
                                            selected_frequencies,
                                            max_freq=max_freq,
                                            buffer_f=buffer_f,
                                            num_frequencies=num_frequencies)

In [ ]:
from plotting_beamform import plot_spectrum_polar

plot_spectrum_polar(mvdr_scan, thetas_scan, frequencies_hz, gt_azimuth=gt_azimuth)
plt.title("MVDR")
plt.legend(loc="upper left", bbox_to_anchor=[1.0, 1.0])

plot_spectrum_polar(das_scan, thetas_scan, frequencies_hz, gt_azimuth=gt_azimuth)
plt.title("DAS")
plt.legend(loc="upper left", bbox_to_anchor=[1.0, 1.0])

## 4.3 Plot and listen to signals

In [ ]:
signals_source = signals["signals_source"].values[0]
signals_all = signals["signals_all"].values[0]
signals_props = signals["signals_props"].values[0]

print("original frequency resolution", Fs / n_times)
times = np.arange(0, 1 / Fs * n_times, step=1 / Fs)

fig, axs = plt.subplots(1, len(filenames), sharey=True, sharex=True)
fig.set_size_inches(len(filenames) * 2.5, 2.5)
for i, ax in enumerate(axs):
    ax.plot(times, signals_props[i], label="propellers")
    ax.plot(times, signals_source[i], label="source")
    ax.plot(times, signals_all[i], label="combined")
    ax.set_xlabel("time [s]")
    ax.set_title(filenames[i])
    ax.legend(loc="lower left")

# ipd.Audio(data=signals_all[0], rate=Fs, normalize=True)
# ipd.Audio(data=signals_source[3], rate=Fs, normalize=True)
ipd.Audio(data=signals_props[3], rate=Fs, normalize=True)

## 4.4 listen to results

In [ ]:
# apply mvdr or delay and sum beamformer, respectively
buffer_f = np.fft.rfft(buffer, n=n_buffer, axis=1)
buffer_f_small = buffer_f[:, frequency_bins]

# filtered output
y_das_f_small = np.sum(np.multiply(H_das.T, buffer_f_small), axis=0)
y_das_f = np.zeros((buffer_f.shape[1],), dtype=np.complex128)
y_das_f[frequency_bins] = y_das_f_small
y_das = np.fft.irfft(y_das_f, n=n_buffer)

y_mvdr_f_small = np.sum(np.multiply(H_mvdr.T, buffer_f_small), axis=0)
y_mvdr_f = np.zeros((buffer_f.shape[1],), dtype=np.complex128)
y_mvdr_f[frequency_bins] = y_mvdr_f_small
y_mvdr = np.fft.irfft(y_mvdr_f, n=n_buffer)

# nonfiltered output
y_f_small = np.sum(buffer_f_small, axis=0)
y_f = np.zeros((buffer_f.shape[1],), dtype=np.complex128)
y_f[frequency_bins] = y_f_small
y = np.fft.irfft(y_f, n=n_buffer)

In [ ]:
plt.figure()
plt.plot(times_buffer, y, label="original")
plt.plot(times_buffer, y_das, linestyle=":", label="DAS")
plt.plot(times_buffer, y_mvdr, linestyle=":", label="MVDR")
plt.legend()

plt.figure()
plt.loglog(np.abs(y_f), label="original")
plt.loglog(np.abs(y_das_f), linestyle=":", label="DAS")
plt.loglog(np.abs(y_mvdr_f), linestyle=":", label="MVDR")
plt.legend()

In [ ]:
ipd.Audio(data=y, rate=Fs, normalize=True)